https://gist.github.com/spro/ef26915065225df65c1187562eca7ec4

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
import numpy as np
import math, random

from bokeh.plotting import figure, show, output_notebook

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
# Generating a noisy multi-sin wave 
class noisymultisin():
    def __init__(self):
        self.offset = 0
        
    def sine_2(self, X, signal_freq=60.):
        return (np.sin(2 * np.pi * (X) / signal_freq) + np.sin(4 * np.pi * (X) / signal_freq)) / 2.0

    def noisy(self, Y, noise_range=(-0.05, 0.05)):
        noise = np.random.uniform(noise_range[0], noise_range[1], size=Y.shape)
        return Y + noise

    def sample(self, sample_size):
        X = np.arange(sample_size)
        X += self.offset
        Y = self.noisy(self.sine_2(X + self.offset))
        self.offset += sample_size
        return X, Y

In [3]:
# generating a simple sin wave
class simplesin():
    def __init__(self):
        self.offset = 0
        
    def sin(self, X, signal_freq=60.):
        return np.sin(2 * np.pi * (X) / signal_freq)

    def sample(self, sample_size):
        X = np.arange(sample_size)
        X += self.offset
        Y = self.sin(X)
        self.offset += sample_size
        return X, Y

In [4]:
# Define the model

class SimpleRNN(nn.Module):
    def __init__(self, hidden_size, dropout=0.05, num_layers=2):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout=dropout
        self.num_layers=num_layers

        self.inp = nn.Linear(1, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers=self.num_layers, dropout=self.dropout)
        self.out = nn.Linear(hidden_size, 1)

    def printParams(self, index=None):
        if index is None:
            print("\nPrinting parameters")
            for parameter in self.parameters():
                print(parameter)
        else:
            for i, value in enumerate(self.parameters()):
                if i == index:
                    print(f"\nPrinting parameters at index {i}")
                    print(value)
                    break
    
    def step(self, input, hidden=None):
        input = self.inp(input.view(1, -1)).unsqueeze(1)
        output, hidden = self.rnn(input, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

    def forward(self, inputs, hidden=None, force=True, steps=0):
        if force or steps == 0: 
            steps = len(inputs)
        outputs = torch.zeros(steps, 1, 1).to(device)
        for i in range(steps):
            if force or i == 0:
                input = inputs[i]
            else:
                input = output
            output, hidden = self.step(input, hidden)
            outputs[i] = output

        return outputs, hidden

Notes: the addition of teach forcing really improve the capacity of the model to genenerate the sinwave by itself

In [5]:
n_epochs = 50
hidden_size = 10
dropout=0.00
num_layers=2

model = SimpleRNN(hidden_size=hidden_size, dropout=dropout, num_layers=num_layers)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.3)

losses = np.zeros(n_epochs) # For plotting

wave = simplesin()
timesteps, _inputs = wave.sample(sample_size=1000)
inputs = torch.from_numpy(_inputs[:-1]).float().to(device)
targets = torch.from_numpy(_inputs[1:]).float().to(device)

for epoch in range(n_epochs):
 
    optimizer.zero_grad()

    # teach forcing
    force = np.random.rand()
    outputs, hidden = model(inputs, force=force)

    loss = criterion(outputs.view(len(outputs)), targets)
    loss.backward()
    optimizer.step()
    losses[epoch] += loss.item()
    
    # print 10 steps of the training and the last one 
    if (epoch % math.floor(n_epochs/10) == 0) or (epoch == n_epochs-1):
        print(f"{epoch+1}: {loss.item()}")

1: 0.5351684093475342
6: 0.4633595645427704
11: 0.37539154291152954
16: 0.16418710350990295
21: 0.09719697386026382
26: 0.07901448756456375
31: 0.06676063686609268
36: 0.05765816569328308
41: 0.05047355219721794
46: 0.044533077627420425
50: 0.04039423540234566


In [6]:
output_notebook()

l = figure(plot_width=900, plot_height=400)
l.line(x=range(n_epochs), y=losses, line_width=2, line_color="blue")
l.xaxis.axis_label = "Epoch"
l.yaxis.axis_label = "Loss"
l.yaxis.major_label_orientation = "vertical"
show(l)

Loading BokehJS ...

In [8]:
output_notebook()

p = figure(plot_width=900, plot_height=400)
p.line(x=timesteps, y=_inputs, line_width=2, line_color="blue", legend="True")
p.line(x=timesteps[1:], y=outputs.tolist(), line_width=2, line_color="orange", legend="Prediction")
show(p)

Loading BokehJS ...


Try to do some prediction by taking more points in the sin wave


In [9]:
timesteps2, _inputs2 = wave.sample(sample_size=100)
inputs2 = torch.from_numpy(_inputs2[:-1]).float()
targets2 = torch.from_numpy(_inputs2[1:]).float()
outputs2, _ = model(inputs2, force=False)

In [10]:
output_notebook()

p = figure(plot_width=900, plot_height=400, title="Generation of the waves by the model without any correction")

# add a line renderer
p.line(x=timesteps2, y=_inputs2, line_width=2, line_color="blue")
p.line(x=timesteps2[1:], y=outputs2.tolist(), line_width=2, line_color="orange")
show(p)

Loading BokehJS ...

Things to try
- Finf the lr with FastAI tricks
- Train for a lot of epoch in lab